### Setup MLRun Project

In [17]:
from os import path
from mlrun import set_environment, new_project, mlconf

# Set the default environment configuration
project_name_base = "igz-mnist-demo"
project_name, artifact_path = set_environment(project=project_name_base,
                                              artifact_path="/User/artifacts/{{run.project}}/{{workflow.uid}}")

# Create project
project_path = path.abspath("project")
project = new_project(name=project_name, context=project_path)

print(f"Project name: {project_name}\nProject path: {project_path}")
print(f"Artifacts path: {artifact_path}\nMLRun DB path: {mlconf.dbpath}")

Project name: igz-mnist-demo
Project path: /User/igz_repos/igz-mnist-demo/project
Artifacts path: /User/artifacts/{{run.project}}/{{workflow.uid}}
MLRun DB path: http://mlrun-api:8080


### Build Docker Image

In [18]:
image = "mlrun/ml-models"

In [19]:
# # Build Docker Image (only needs to be run once)
# from mlrun import new_function

# build_image = new_function(name="build-image", kind="job")
# build_image.build_config(
#     image=image,
#     base_image="mlrun/ml-models",
#     commands=["pip install storey"]
# )
# build_image.deploy(with_mlrun=False)

In [20]:
image

'mlrun/ml-models'

### Import Functions

In [21]:
project.set_function(func="components/get_prep_data.py",
                     name="get-prep-data",
                     kind="job",
                     image=image)
project.set_function(func="components/train_eval_model.py",
                     name="train-eval-model",
                     kind="job",
                     image=image)
project.set_function(func="components/deploy_model.py",
                     name="deploy-model",
                     kind="serving",
                     image=image)

### Pipeline

In [22]:
%%writefile {path.join(project_path, 'pipeline.py')}

import os
from kfp import dsl
from mlrun import mount_v3io, NewTask
import yaml
import nuclio

funcs = {}

# Configure function resources and local settings
def init_functions(functions: dict, project=None, secrets=None):
    for fn in functions.values():
        # Mount V3IO filesystem
        fn.apply(mount_v3io())

    functions["deploy-model"].spec.base_spec['spec']['loggerSinks'] = [{'level': 'info'}]
    functions["deploy-model"].spec.min_replicas = 1
    functions["deploy-model"].spec.max_replicas = 1
    functions["deploy-model"].spec.default_class = 'MNISTModel'

# Create a Kubeflow Pipelines pipeline
@dsl.pipeline(
    name="MNIST Digit Classification Pipeline",
    description="Kubeflow Pipeline Demo to classify Handwritten Digits"
)
def kfpipeline(batch_size:int=128,
               num_classes:int=10,
               epochs:int=5,
               debug_logs:bool=True):    
    
    # Get data from feature store, prep, train/test split
    get_prep_data = funcs['get-prep-data'].as_step(
        handler="handler",
        outputs=["X_train", "X_test", "y_train", "y_test"],
        verbose=debug_logs)
    
    # Train/evaluate model
    train = funcs['train-eval-model'].as_step(
        handler="handler",
        inputs={"X_train" : get_prep_data.outputs['X_train'],
                "X_test" : get_prep_data.outputs['X_test'],
                "y_train" : get_prep_data.outputs['y_train'],
                "y_test" : get_prep_data.outputs['y_test']},
        params={"batch_size" : batch_size,
                "num_classes" : num_classes,
                "epochs" : epochs},
        outputs=["model"],
        verbose=debug_logs)
    
    # Deploy model
    deploy = funcs["deploy-model"].deploy_step(models=[{"key": "model",
                                                        "model_path": train.outputs['model']}])

Overwriting /User/igz_repos/igz-mnist-demo/project/pipeline.py


### Save Pipeline

In [23]:
project.set_workflow("main", "pipeline.py")
project.save()

### Pipeline

In [11]:
run_id = project.run("main", arguments={}, artifact_path=artifact_path, dirty=True)

> 2021-03-08 17:49:20,235 [info] Pipeline run id=ee1252fe-3072-489e-94d8-60bd2aed22bc, check UI or DB for progress


### Inference Model

In [29]:
import json
import requests
import numpy as np

addr = 'http://nuclio-{}-{}:8080'.format(project.name, project.func('deploy-model').metadata.name)
addr += "/v2/models/model/predict"
addr

'http://nuclio-igz-mnist-demo-deploy-model:8080/v2/models/model/predict'

In [30]:
import os
import sys
import base64
from io import BytesIO
from PIL import Image
import json
from tensorflow.keras.datasets import mnist

def encode_image(img):
    im_file = BytesIO()
    img.save(im_file, format="JPEG")
    im_bytes = im_file.getvalue()
    im_b64 = base64.b64encode(im_bytes)
    return im_b64.decode("utf-8")

(X_train, y_train), (X_test, y_test) = mnist.load_data()
img = Image.fromarray(X_test[2])
img

In [31]:
payload = json.dumps({"inputs" : [encode_image(img)]})
payload

'{"inputs": ["/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APn+lALEAAkngAd62PFGgjw1rR0trrz5o4Ynn/d7PLkZAzR9TnbnGfboKxqK634a6PHrPjrT1uAhs7Mm9ut/TyovmOfrgD8awte1M614h1LVCGX7ZcyThWOSoZiQM+2cVn0V6J4RFvo/wu8X63NKi3F6qaVarxuYsQ0nvjaR+XevO6KKKKK//9k="]}'

In [32]:
response = requests.post(addr, json=payload)
predictions = response.json()

In [33]:
digit, acc = predictions['outputs']
print(f"Predicted the digit {digit} with {acc} accuracy.")

Predicted the digit 1 with 0.11261995136737823 accuracy.
